In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-131191")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131191
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-131191


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for the CPU cluster
cpu_cluster_name= "cpu-cluster-4"

#verify cluster doesn't already exist
try:
    compute_target= ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=6)
    compute_target= ComputeTarget.create(ws, cpu_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)



# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice, loguniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.2, 0.4, 1,2),
        '--max_iter': choice (50, 100, 150, 200)
    }
)

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =SKLearn(source_directory='.',
entry_script='train.py',
compute_target=cpu_cluster_name)

from azureml.core.script_run_config import ScriptRunConfig

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(estimator=est,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name="Accuracy",
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=100,
max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.core.experiment import Experiment
experiment=Experiment(ws, ws.name)
hyperdrive_run=experiment.submit(hyperdrive_config, show_output=True)



In [8]:
import joblib
from azureml.core.model import Model
# Get your best run and save the model from that run.
os.makedirs('outputs', exist_ok=True)


best_run=hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=best_run, filename='outputs/model.pkl')
model=Model.register(workspace=ws,model_name='hyperdrive_model',
                             model_path='outputs/model.pkl',                     
model_framework=Model.Framework.SCIKITLEARN, model_framework_version='0.19.1')


Registering model hyperdrive_model


In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core  import Dataset 
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
#ds = pd.read_csv("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")  ### YOUR CODE HERE ###
ds=Dataset.Tabular.from_delimited_files(path=url)

In [17]:
#from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###

NameError: name 'clean_data' is not defined

In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds,
    label_column_name='POSITION',
    n_cross_validations=5)

In [20]:
# Submit your automl run
from azureml.core.experiment import Experiment
experiment=Experiment(ws, "automl_test_experiment")
run=experiment.submit(config=automl_config, show_output=True)

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

run.complete()

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Package name/Required version/Installed version
azureml-core/pyopenssl<20.0.0/pyOpenSSL 20.0.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nPackage name/Required version/Installed version\nazureml-core/pyopenssl<20.0.0/pyOpenSSL 20.0.0",
        "target": "azureml-core",
        "inner_error": {
            "code": "NotSupported",
     

In [ ]:
# Retrieve and save your best automl model.

best_run=hyperdrive_run.get_best_run_bu_primary_metrix()
print(best_run.get_details()['runDefinition']['arguments'])
print(nest_run.get_file_names())